In [2]:
# libraries import
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

In [4]:
# read gff file and data structuring
contigs = {}

with open('GCF_003789085.1_ASM378908v1_genomic.gff', 'r') as f_input:
    for line in f_input:
        if line.startswith('#') or not line.strip():
            continue

        parts = line.strip().split('\t')

        contig_id = parts[0]
        feature_type = parts[2]
        start = int(parts[3])
        end = int(parts[4])
        strand = parts[6]
        metadata = parts[8]
        metadata_split = metadata.split(';')
        metadata_dict = {item.split('=')[0]: item.split('=')[1] for item in metadata_split if '=' in item}

        feature = {
            'type': feature_type,
            'start': start,
            'end': end,
            'strand': strand,
            'metadata': metadata_dict,
        }

        if contig_id not in contigs:
            contigs[contig_id] = []
        contigs[contig_id].append(feature)

# check for data structure (not necessary)
# for contig_id, features in contigs.items():
#     print(f"Contig: {contig_id}, Number of features: {len(features)}")
#     for feature in features[:5]:  # display the first 5 features of each contig
#         print(feature)
#     print()

In [5]:
# count the long non-coding RNAs in each contig 
lncRNA_count_per_contig = defaultdict(set)

for contig_id, features in contigs.items():
    for feature in features:
        if feature['type'] == 'lnc_RNA':
            # Use the unique ID directly to count lncRNAs
            gene_id = feature['metadata'].get('ID', 'Unknown')
            if gene_id != 'Unknown':
                lncRNA_count_per_contig[contig_id].add(gene_id)

# Display the number of long non-coding RNAs per contig
total_lncRNAs = sum(len(lncRNAs) for lncRNAs in lncRNA_count_per_contig.values())
print(f"Total long non-coding RNAs: {total_lncRNAs}")
for contig_id, lncRNAs in lncRNA_count_per_contig.items():
    print(f"Contig: {contig_id}, Number of unique lncRNAs: {len(lncRNAs)}")

# Verify if the total matches the expected 2,028
if total_lncRNAs == 2028:
    print("The total number of long non-coding RNAs matches the expected: 2,028")
else:
    print(f"The total number of long non-coding RNAs ({total_lncRNAs}) does not match the expected: 2,028")

Total long non-coding RNAs: 2028
Contig: NW_020868287.1, Number of unique lncRNAs: 1
Contig: NW_020868290.1, Number of unique lncRNAs: 6
Contig: NW_020868291.1, Number of unique lncRNAs: 1
Contig: NW_020868292.1, Number of unique lncRNAs: 7
Contig: NW_020868300.1, Number of unique lncRNAs: 1
Contig: NW_020868305.1, Number of unique lncRNAs: 1
Contig: NW_020868307.1, Number of unique lncRNAs: 16
Contig: NW_020868311.1, Number of unique lncRNAs: 2
Contig: NW_020868314.1, Number of unique lncRNAs: 1
Contig: NW_020868326.1, Number of unique lncRNAs: 2
Contig: NW_020868327.1, Number of unique lncRNAs: 1
Contig: NW_020868328.1, Number of unique lncRNAs: 1
Contig: NW_020868329.1, Number of unique lncRNAs: 3
Contig: NW_020868337.1, Number of unique lncRNAs: 1
Contig: NW_020868347.1, Number of unique lncRNAs: 4
Contig: NW_020868350.1, Number of unique lncRNAs: 1
Contig: NW_020868353.1, Number of unique lncRNAs: 2
Contig: NW_020868355.1, Number of unique lncRNAs: 1
Contig: NW_020868364.1, Number

In [6]:
# load the IDs in a .txt file as a list
ids_interes = set()
with open('true_Up_muscle_lncRNAs.txt', 'r') as f_ids:
    for line in f_ids:
        ids_interes.add(line.strip())

print("IDs de interés cargados:", ids_interes)  # Mensaje de depuración

# Encontrar en qué contigs están presentes los IDs de interés
contigs_interes = defaultdict(list)

for contig_id, features in contigs.items():
    for feature in features:
        if feature['type'] == 'lnc_RNA':
            metadata_dict = feature['metadata']
            gene_id = metadata_dict.get('ID', 'Unknown')
            if gene_id.startswith('rna-'):
                gene_id = gene_id[4:]  # Remover el prefijo 'rna-'
            if gene_id in ids_interes:
                contigs_interes[gene_id].append(contig_id)

# Mostrar los resultados
# for gene_id, contig_list in contigs_interes.items():
#     print(f"ID: {gene_id}, Contigs: {contig_list}")

IDs de interés cargados: {'XR_003476908.1', 'XR_003476489.1', 'XR_003477015.1', 'XR_003477013.1', 'XR_003476309.1', 'XR_003475609.1', 'XR_003476045.1', 'XR_003477469.1', 'XR_003477805.1', 'XR_003476046.1', 'XR_003476044.1', 'XR_003477476.1'}
ID: XR_003475609.1, Contigs: ['NW_020868904.1']
ID: XR_003476046.1, Contigs: ['NW_020869391.1']
ID: XR_003476045.1, Contigs: ['NW_020869391.1']
ID: XR_003476044.1, Contigs: ['NW_020869391.1']
ID: XR_003476309.1, Contigs: ['NW_020869751.1']
ID: XR_003476489.1, Contigs: ['NW_020869983.1']
ID: XR_003476908.1, Contigs: ['NW_020870418.1']
ID: XR_003477013.1, Contigs: ['NW_020870513.1']
ID: XR_003477015.1, Contigs: ['NW_020870513.1']
ID: XR_003477469.1, Contigs: ['NW_020871061.1']
ID: XR_003477476.1, Contigs: ['NW_020871071.1']
ID: XR_003477805.1, Contigs: ['NW_020872368.1']


In [7]:
# define funciton to search for neighbouring mRNAs 
def find_mRNA_neighbors(features, lncRNA_index):
    """
    search neighbouring mRNAs for a given lncRNA
    return upstream and downstream mRNA indices if they exist
    """
    upstream_mRNA = downstream_mRNA = None
    lncRNA = features[lncRNA_index]

    for i, feature in enumerate(features):
        if feature['type'] == 'mRNA':
            if feature['end'] < lncRNA['start']:  # upstream mRNA
                if upstream_mRNA is None or features[upstream_mRNA]['end'] < feature['end']:
                    upstream_mRNA = i
            elif feature['start'] > lncRNA['end']:  # downstream mRNA
                if downstream_mRNA is None or features[downstream_mRNA]['start'] > feature['start']:
                    downstream_mRNA = i

    return upstream_mRNA, downstream_mRNA

# search for neighbouring mRNAs for a interest given IDs
for gene_id in ids_interes:
    for contig_id in contigs_interes[gene_id]:
        features = contigs[contig_id]
        for i, feature in enumerate(features):
            if feature['type'] == 'lnc_RNA' and feature['metadata'].get('ID', '').endswith(gene_id):
                upstream_mRNA_idx, downstream_mRNA_idx = find_mRNA_neighbors(features, i)
                if upstream_mRNA_idx is not None and downstream_mRNA_idx is not None:
                    upstream_mRNA = features[upstream_mRNA_idx]
                    downstream_mRNA = features[downstream_mRNA_idx]

                    upstream_mRNA_id = upstream_mRNA['metadata'].get('ID', 'Unknown')
                    downstream_mRNA_id = downstream_mRNA['metadata'].get('ID', 'Unknown')

                    print(f"lncRNA {gene_id} in contig {contig_id}:")
                    print(f"  Upstream mRNA: {upstream_mRNA_id}")
                    print(f"  Downstream mRNA: {downstream_mRNA_id}")
                    
                    # check and report overlappings
                    if feature['start'] <= upstream_mRNA['end']:
                        print(f"  Overlaps with upstream mRNA: {upstream_mRNA_id}")
                    if feature['end'] >= downstream_mRNA['start']:
                        print(f"  Overlaps with downstream mRNA: {downstream_mRNA_id}")
                    print() 
                else:
                    if upstream_mRNA_idx is None:
                        print(f"  No upstream mRNA found for lncRNA {gene_id} in contig {contig_id}\n")
                    if downstream_mRNA_idx is None:
                        print(f"  No downstream mRNA found for lncRNA {gene_id} in contig {contig_id}\n")

lncRNA XR_003476908.1 in contig NW_020870418.1:
  Upstream mRNA: rna-XM_027370782.1
  Downstream mRNA: rna-XM_027370783.1

  No upstream mRNA found for lncRNA XR_003476489.1 in contig NW_020869983.1

lncRNA XR_003477015.1 in contig NW_020870513.1:
  Upstream mRNA: rna-XM_027371589.1
  Downstream mRNA: rna-XM_027371590.1

lncRNA XR_003477013.1 in contig NW_020870513.1:
  Upstream mRNA: rna-XM_027371589.1
  Downstream mRNA: rna-XM_027371590.1

  No upstream mRNA found for lncRNA XR_003476309.1 in contig NW_020869751.1

lncRNA XR_003475609.1 in contig NW_020868904.1:
  Upstream mRNA: rna-XM_027355539.1
  Downstream mRNA: rna-XM_027355540.1

lncRNA XR_003476045.1 in contig NW_020869391.1:
  Upstream mRNA: rna-XM_027360592.1
  Downstream mRNA: rna-XM_027360584.1

lncRNA XR_003477469.1 in contig NW_020871061.1:
  Upstream mRNA: rna-XM_027376763.1
  Downstream mRNA: rna-XM_027376772.1

lncRNA XR_003477805.1 in contig NW_020872368.1:
  Upstream mRNA: rna-XM_027381290.1
  Downstream mRNA: rna-X

In [8]:
################ Calculate percentage of lncRNAs with neighboring mRNAs ################
def find_mRNA_neighbors(features, lncRNA_index):
    """
    search neighbouring mRNAs for a given lncRNA
    return upstream and downstream mRNA indices if they exist
    """
    upstream_mRNA = downstream_mRNA = None
    lncRNA = features[lncRNA_index]

    for i, feature in enumerate(features):
        if feature['type'] == 'mRNA':
            if feature['end'] < lncRNA['start']:  # upstream mRNA
                if upstream_mRNA is None or features[upstream_mRNA]['end'] < feature['end']:
                    upstream_mRNA = i
            elif feature['start'] > lncRNA['end']:  # downstream mRNA
                if downstream_mRNA is None or features[downstream_mRNA]['start'] > feature['start']:
                    downstream_mRNA = i

    return upstream_mRNA, downstream_mRNA

# percentage of lncRNAs with neighboring mRNAs
total_lncRNAs = 0
flanked_lncRNAs = 0

for contig_id, features in contigs.items():
    for i, feature in enumerate(features):
        if feature['type'] == 'lnc_RNA':
            total_lncRNAs += 1
            upstream_mRNA_idx, downstream_mRNA_idx = find_mRNA_neighbors(features, i)
            if upstream_mRNA_idx is not None and downstream_mRNA_idx is not None:
                flanked_lncRNAs += 1

# calculate
if total_lncRNAs > 0:
    percentage_flanked = (flanked_lncRNAs / total_lncRNAs) * 100
    print(f"Total lncRNAs: {total_lncRNAs}")
    print(f"Flanked lncRNAs: {flanked_lncRNAs}")
    print(f"Percentage of flanked lncRNAs: {percentage_flanked:.2f}%")
else:
    print("No lncRNAs found.")

Total lncRNAs: 2028
Flanked lncRNAs: 1512
Percentage of flanked lncRNAs: 74.56%


In [9]:
################ calculate distances between up/downstream ################
def find_mRNA_neighbors(features, lncRNA_index):
    """
    search neighbouring mRNAs for a given lncRNA
    return upstream and downstream mRNA indices if they exist
    """
    upstream_mRNA = downstream_mRNA = None
    lncRNA = features[lncRNA_index]

    for i, feature in enumerate(features):
        if feature['type'] == 'mRNA':
            if feature['end'] < lncRNA['start']:  # upstream mRNA
                if upstream_mRNA is None or features[upstream_mRNA]['end'] < feature['end']:
                    upstream_mRNA = i
            elif feature['start'] > lncRNA['end']:  # downstream mRNA
                if downstream_mRNA is None or features[downstream_mRNA]['start'] > feature['start']:
                    downstream_mRNA = i

    return upstream_mRNA, downstream_mRNA

# distances calculations
total_lncRNAs = 0
flanked_lncRNAs = 0
upstream_distances = []
downstream_distances = []

for contig_id, features in contigs.items():
    for i, feature in enumerate(features):
        if feature['type'] == 'lnc_RNA':
            total_lncRNAs += 1
            upstream_mRNA_idx, downstream_mRNA_idx = find_mRNA_neighbors(features, i)
            if upstream_mRNA_idx is not None and downstream_mRNA_idx is not None:
                flanked_lncRNAs += 1
                lncRNA = feature 
                upstream_mRNA = features[upstream_mRNA_idx]
                downstream_mRNA = features[downstream_mRNA_idx]

                upstream_distance = lncRNA['start'] - upstream_mRNA['end']
                downstream_distance = downstream_mRNA['start'] - lncRNA['end']
                upstream_distances.append(upstream_distance)
                downstream_distances.append(downstream_distance)

# disances mean
if upstream_distances:
    avg_upstream_distance = sum(upstream_distances) / len(upstream_distances) / 1000  # to Kb
else:
    avg_upstream_distance = 0

if downstream_distances:
    avg_downstream_distance = sum(downstream_distances) / len(downstream_distances) / 1000  # to Kb
else:
    avg_downstream_distance = 0

# print results
print(f"Total lncRNAs: {total_lncRNAs}")
print(f"Flanked lncRNAs: {flanked_lncRNAs}")
print(f"Percentage of flanked lncRNAs: {flanked_lncRNAs / total_lncRNAs * 100:.2f}%")
print(f"Average upstream distance: {avg_upstream_distance:.2f} kb")
print(f"Average downstream distance: {avg_downstream_distance:.2f} kb")

Total lncRNAs: 2028
Flanked lncRNAs: 1512
Percentage of flanked lncRNAs: 74.56%
Average upstream distance: 34.35 kb
Average downstream distance: 35.36 kb


In [10]:
########################### MAIN FUNCTION ###########################
def find_mRNA_neighbors(features, lncRNA_index):
    """
    search neighbouring mRNAs for a given lncRNA
    return upstream and downstream mRNA indices if they exist
    """
    upstream_mRNA = downstream_mRNA = None
    lncRNA = features[lncRNA_index]

    for i, feature in enumerate(features):
        if feature['type'] == 'mRNA':
            if feature['end'] < lncRNA['start']:  # upstream mRNA
                if upstream_mRNA is None or features[upstream_mRNA]['end'] < feature['end']:
                    upstream_mRNA = i
            elif feature['start'] > lncRNA['end']:  # downstream mRNA
                if downstream_mRNA is None or features[downstream_mRNA]['start'] > feature['start']:
                    downstream_mRNA = i

    return upstream_mRNA, downstream_mRNA

# distances calculations
lncRNA_distances = []

for gene_id in ids_interes:
    for contig_id in contigs_interes[gene_id]:
        features = contigs[contig_id]
        for i, feature in enumerate(features):
            if feature['type'] == 'lnc_RNA' and feature['metadata'].get('ID', '').endswith(gene_id):
                upstream_mRNA_idx, downstream_mRNA_idx = find_mRNA_neighbors(features, i)
                if upstream_mRNA_idx is not None and downstream_mRNA_idx is not None:
                    upstream_mRNA = features[upstream_mRNA_idx]
                    downstream_mRNA = features[downstream_mRNA_idx]

                    upstream_distance = feature['start'] - upstream_mRNA['end']
                    downstream_distance = downstream_mRNA['start'] - feature['end']
                    lncRNA_distances.append({
                        'lncRNA_id': gene_id,
                        'contig_id': contig_id,
                        'upstream_mRNA_id': upstream_mRNA['metadata'].get('ID', 'Unknown'),
                        'downstream_mRNA_id': downstream_mRNA['metadata'].get('ID', 'Unknown'),
                        'upstream_distance': upstream_distance,
                        'downstream_distance': downstream_distance
                    })

# print results
for entry in lncRNA_distances:
    print(f"lncRNA ID: {entry['lncRNA_id']} in contig {entry['contig_id']}:")
    print(f"  Upstream mRNA ID: {entry['upstream_mRNA_id']}, Distance: {entry['upstream_distance']} bases")
    print(f"  Downstream mRNA ID: {entry['downstream_mRNA_id']}, Distance: {entry['downstream_distance']} bases")
    print()

lncRNA ID: XR_003476908.1 in contig NW_020870418.1:
  Upstream mRNA ID: rna-XM_027370782.1, Distance: 12358 bases
  Downstream mRNA ID: rna-XM_027370783.1, Distance: 2656 bases

lncRNA ID: XR_003477015.1 in contig NW_020870513.1:
  Upstream mRNA ID: rna-XM_027371589.1, Distance: 33033 bases
  Downstream mRNA ID: rna-XM_027371590.1, Distance: 72507 bases

lncRNA ID: XR_003477013.1 in contig NW_020870513.1:
  Upstream mRNA ID: rna-XM_027371589.1, Distance: 29098 bases
  Downstream mRNA ID: rna-XM_027371590.1, Distance: 76338 bases

lncRNA ID: XR_003475609.1 in contig NW_020868904.1:
  Upstream mRNA ID: rna-XM_027355539.1, Distance: 50245 bases
  Downstream mRNA ID: rna-XM_027355540.1, Distance: 111264 bases

lncRNA ID: XR_003476045.1 in contig NW_020869391.1:
  Upstream mRNA ID: rna-XM_027360592.1, Distance: 3674 bases
  Downstream mRNA ID: rna-XM_027360584.1, Distance: 28121 bases

lncRNA ID: XR_003477469.1 in contig NW_020871061.1:
  Upstream mRNA ID: rna-XM_027376763.1, Distance: 84 b